<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/rbot_version_3/manual/bybit/bybit_market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bybit用データ蓄積クラスを使ったサンプル

`Bybit`は、通過ペアー毎に以下の機能を提供します。

* 蓄積用データベースの生成（Sqlite)
* 約定データのダウンロードと更新
  * 過去データのダウンロード(Binance Webサイト)
  * リアルタイム更新(WebSocket)
* 約定データのDBからの取得と足の作成
  * 任意の期間の足の作成(`ohlcv`メソッド)
  * VAP(Value At Price)の計算(`vap`メソッド)

また通過ペアーの設定は`BybitConfig`クラスに定義してあります。パラーメータを修正することで任意の通過ペアーに対応可能ですが、テスト完了して提供しているものは以下になります。
* `BybytConfig.BTCUSDT` BTCUSDTペアー（Perpetual）

また注意点としてはDBに書き込めるのは１プロセスのみとなります。複数プロセスが立ち上がっているとエラーになったりロックがかかりフリーズする場合があります。

## 本サンプルの機能

本サンプルでは以下を行います。フレームワークがかなりの部分やってくれるのでシンプルに実行できることがわかると思います。

* BTCUSDTペアーの設定で`BybitMarket`オブジェクトを生成する
* 過去データのダウンロード
* OHLCVの作成とPlotlyによる描画
* VAP(Value At Price)の作成と描画
* WebSocketによるリアルタイムデータの受信（板情報）

## 必要ライブラリのインストール

`rbot`がBotFrameWorkの本体です。必要なライブラリを合わせてインストールします。２回目は実行不要です。

In [1]:
# 必要ライブラリのインストール
! pip install --upgrade pip

# rbotがbot frameworkになります。PyPiに登録されているので、pipでインストールできます。
# まだ開発初期段階なので、インターフェースが変わる可能性があります。
! pip install -i https://test.pypi.org/simple/ rbot
#! pip install --upgrade rbot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Looking in indexes: https://test.pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 10.5 MB/s eta 0:00:00


In [2]:
# 必要ライブラリのインストール。環境によっては要・不要があるので適宜修正してください。
! pip install json2html
! pip install plotly
! pip install nbformat
! pip install pandas
! pip install polars
! pip install pyarrow
! pip install --upgrade ipywidgets
! pip install --upgrade ipympl

  Preparing metadata (setup.py) ... done
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7593 sha256=bf1f05a31cd03c3d47e6cffbc7ddf85e0ed2c036d46d2453024f40d6fd07c167
  Stored in directory: /root/.cache/pip/wheels/e0/d8/b3/6f83a04ab0ec00e691de794d108286bb0f8bcdf4ade19afb57
Successfully built json2html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 7.2 MB/

## Python コード本体

### 必要ライブラリのインポート

In [3]:
import rbot
# BybitMarketクラスは、BybitのAPIを利用して、取引所の情報を取得するクラス
from rbot import Bybit

# BybitConfigクラスは、BinanceMarketクラスのコンストラクタに渡す設定クラス
from rbot import BybitConfig
from rbot import init_log, init_debug_log
#init_debug_log()

print("rbot version =", rbot.__version__)

rbot version = 0.3.2


In [4]:
# 必要ライブラリーのインポート
from json2html import *
from IPython.display import HTML

### BybitMarketオブジェクトの生成

#### Clab向けディレクトリ設定

`RBOT_DB_ROOT`にDBの場所を指定することができます。指定しない場合は、システムディレクトリ(`Library/Application Suport/net.takibi.rbot/DB`など)に作られます。marketオブジェクトの`file_name`プロパティで確認できます。


In [5]:
# Google Colabの場合は、GoogleDriveにDBをつくります。
if 'google.colab' in str(get_ipython()):
    # Google Driveをマウントする
    from google.colab import drive
    drive.mount('/content/drive')

    # DBの保存先をGoogle Driveに変更する
    import os
    os.environ['RBOT_DB_ROOT'] = '/content/drive/MyDrive'    # MyDrive以下を指定しましたが適宜変更ください


Mounted at /content/drive


#### 引数にBybitConfigを設定してオブジェクトを生成します。

In [6]:
# BinanceMarketクラスのインスタンスを生成(BTCUSDT現物取引の設定)
config = BybitConfig.BTCUSDT

# Binanceの本番ネットへ接続する。
exchange  = Bybit(production=True)
market = exchange.open_market(config)

# BinanceMarketのインスタンスを表示すると格納されているデータの情報が表示されます。
market


start,end
0,0
"""1970-01-01T00:00:00.000000""","""1970-01-01T00:00:00.000000"""
days=,0


In [7]:
# DBの場所を確認するには以下のプロパティを読み出す。

# market.file_name

### 約定ログのダウンロード

In [8]:
# 過去１日分のデータをダウンロード。ダウンロードしたデータは、marketオブジェクトの属性に格納される。
# Trueを指定すると、再ダウンロードを行う。Falseの場合はローカルに保存されている場合は、再ダウンロードを行わない。
# 比較的時間がかかる処理です。終わるまで少々お待ちください。完了すると取り込まれたレコード数が表示されます。
market.download_archive(
    ndays=1,        # 過去何日分のデータをダウンロードするか
    force=False,    # Trueを指定すると、再ダウンロードを行う。Falseの場合はローカルに保存されている場合は、再ダウンロードを行わない。
    verbose=True    # Trueを指定すると、ダウンロードの進捗状況を表示する。
)

2134906

In [9]:
# アーカイブ以外のデータを削除する。
market.expire_unfix_data(force=True)

In [10]:
# 直近のデータをダウンロード（1000件)
market.download_latest()

RuntimeError: get_recent_trades: server=BybitServerConfig { production: true, rest_server: "https://api.bybit.com", public_ws: "wss://stream.bybit.com/v5/public", private_ws: "wss://stream.bybit.com/v5/private", history_web_base: "https://public.bybit.com", api_key: ********, api_secret: ******** } / path="/v5/market/recent-trade" / params="category=linear&symbol=BTCUSDT&limit=1000"

Caused by:
    0: rest_get error: https://api.bybit.com//v5/market/recent-trade?category=linear&symbol=BTCUSDT&limit=1000
    1: Response code = 403 / download size Some(986) / method(GET) / URL = https://api.bybit.com/v5/market/recent-trade?category=linear&symbol=BTCUSDT&limit=1000 / path = 

In [ ]:
# ダウンロードされていない最新のブロックGAPをダウンロードする。
# Bybitでは、直近１日の約定データが提供されていないので、Klines（１分足）をつかっています。
# そのため、直近の１日については１分足よりも細かなデータは利用できません。

market.download_gap(verbose=True)

In [11]:
market

start,end
1711497600616600,1711517644074500
"""2024-03-27T00:00:00.616600""","""2024-03-27T05:34:04.074500"""
days=,0


### OHLCVの計算

ohlcvメソッドを使うと任意の時間足でローソク足をつくることができます。戻り値はPolarsのDataFrame型です。

#### `ohlcv`メソッド

```
BybitMarket#ohlcv(
    start_time=0, # 開始時刻(UNIX時間[us]) 0の場合はDBにある最初のレコードから
    end_time=0, # 終了時刻(UNIX時間[us])　 0の場合はDBにある最後のレコードまで
    window_sec=60 #OHLCV足の時間幅(秒)
)
```


In [12]:
# データベースすべての期間,１分足でOHLCVを計算する。
# 初回はメモリーにロードするために時間がかかります。
ohlcv = market.ohlcv(
    start_time=0,
    end_time=0,
    window_sec=60
)

In [13]:
# ohlcvはpolarsのDataFrameオブジェクトとして格納されている。
ohlcv.head()

timestamp,open,high,low,close,volume,count
datetime[μs],f64,f64,f64,f64,f64,u32
2024-03-27 00:00:00,70046.0,70064.4,70035.3,70035.4,66.98,733
2024-03-27 00:01:00,70035.3,70074.7,70035.3,70051.8,54.879,739
2024-03-27 00:02:00,70051.7,70051.8,70022.8,70051.7,39.026,437
2024-03-27 00:03:00,70051.8,70097.7,70051.7,70087.3,42.186,476
2024-03-27 00:04:00,70087.3,70087.3,70041.5,70069.9,41.548,521


### (参考)Pandasへの変換

Polarsの情報はまだ不足気味。また他のライブラリとの互換性の問題でPandasを使いたい場合は、Polarsのデータフレームオブジェクトの`to_pandas`メソッドで簡単にPandasへ変換することができます。このとき`use_pyarrow_extension_array`を`True`に指定するとデータのコピーが発生せず高速です。

In [14]:
# polarsからpandasへ変換する。
pd_ohlcv = ohlcv.to_pandas(use_pyarrow_extension_array=True)

In [15]:
pd_ohlcv.head()

,timestamp,open,high,low,close,volume,count
0,2024-03-27 00:00:00,70046.0,70064.4,70035.3,70035.4,66.98000000000008,733
1,2024-03-27 00:01:00,70035.3,70074.7,70035.3,70051.8,54.87900000000007,739
2,2024-03-27 00:02:00,70051.7,70051.8,70022.8,70051.7,39.026000000000025,437
3,2024-03-27 00:03:00,70051.8,70097.7,70051.7,70087.3,42.185999999999986,476
4,2024-03-27 00:04:00,70087.3,70087.3,70041.5,70069.9,41.54800000000003,521


## Plotlyでローソク足を表示する。

Plotlyを使うと簡単にローソク足を表示できます。polarsのデータそのままで表示可能です。

In [16]:
# OHLCVの表示
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_width=[0.2, 0.7])

fig.add_trace(
        go.Candlestick(
            x=ohlcv['timestamp'],
            open=ohlcv['open'],
            high=ohlcv['high'],
            low=ohlcv['low'],
            close=ohlcv['close'],

        ),
        row=1, col=1
)

fig.add_trace(
        go.Bar(
            x=ohlcv['timestamp'],
            y=ohlcv['volume'],
        ),
        row=2, col=1
)

#fig.update_layout(layout_xaxis_rangeslider_visible=False)

fig


### VAP(Volume At Price)の計算

`BinanceMarket#vap`で指定した期間のVAPを計算します。

```
BinanceMarket#vap(
    start_time = 0, # start_time: 開始時刻(UNIX時間[us]) 0の場合はDBにある最初のレコードから
    end_time = 0, # end_time:終了時刻(UNIX時間[us]) 0の場合はDBの最終レコードまで
    price_unit = 1 #価格をまとめる単位
)
```

In [17]:
vap = market.vap(start_time=0, end_time=0, price_unit=1)

In [18]:
vap

price,buy_volume,sell_volume,volume
f64,f64,f64,f64
69889.0,3.034,0.306,3.34
69890.0,0.0,0.92,0.92
69891.0,0.002,0.085,0.087
69892.0,6.778,4.999,11.777
69893.0,0.701,0.339,1.04
69894.0,0.0,0.248,0.248
69895.0,0.0,0.026,0.026
69896.0,5.37,0.088,5.458
69897.0,1.279,1.755,3.034


In [19]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=vap['sell_volume'],
            y=vap['price'],
            fill='tozerox',
            name='sell'
        ),
        go.Scatter(
            x=vap['sell_volume'] + vap['buy_volume'],
            y=vap['price'],
            fill='tonextx',
            name='buy'
        ),
    ],
    layout=go.Layout(barmode='stack')
)

fig

## WebSocketによるリアルタイム情報取得
(注：ColabではWebSocketが動きません。ローカル環境で試してください）

なお、同時にTick（約定）情報もDBへリアルタイムに更新されていきます。
そのため２回目にOHLCV取得すると、リアルタイム分とバッチダウンロード分の間にギャップができます。
ここを埋めるAPIもあるのですが、また別途。

In [ ]:
# WebSocketデータの受信開始
market.start_market_stream()

from time import sleep
sleep(1)

In [ ]:

from time import sleep
import plotly.graph_objects as go

def trim(bids, asks):
    ask_spred =  asks['price'][-1] - asks['price'][0]
    bids_spred = bids['price'][0] - bids['price'][-1]

    if ask_spred < bids_spred:
        edge = bids['price'][0]
        bids = bids.filter(bids['price'] > edge - ask_spred * 3)
    else:
        edge = asks['price'][0]
        asks = asks.filter(asks['price'] < edge + bids_spred * 3)

    return bids, asks

fig = go.FigureWidget()

bids, asks = market.board
bids, asks = trim(bids, asks)

fig.add_scatter(
    x=bids['price'],
    y=bids['sum'],
    name='bids',
    marker_color='blue'
)

fig.add_scatter(
    x=asks['price'],
    y=asks['sum'],
    name='asks',
    marker_color='red'
)

fig.layout.title = 'Bybit BTCUSDT Order Book'
fig

In [ ]:
# 0.1秒ごとに板情報を更新する（１００回ループ）

for i in range(1000):
    bids, asks = market.board
    bids, asks = trim(bids, asks)
    bids_edge = bids['price'][0]
    asks_edge = asks['price'][0]

    fig.layout.title = f'Bybit BTCUSDT Order Book (bids_edge={bids_edge}, asks_edge={asks_edge})'

    fig.data[0].x = bids['price']
    fig.data[0].y = bids['sum']
    fig.data[1].x = asks['price']
    fig.data[1].y = asks['sum']
    sleep(0.1)






# 以後テスト用

In [ ]:
# VAP内の縦横計算の確認

vap_sum = vap.sum()

print(vap_sum)

vap_sell_sum = vap_sum['sell_volume'][0]
vap_buy_sum = vap_sum['buy_volume'][0]
vap_volume_sum = vap_sum['volume'][0]

print('vap_sell_volume: ', vap_sell_sum)
print('vap_buy_volume: ', vap_buy_sum)
print('vap_volume: ', vap_volume_sum)

if vap_sell_sum + vap_buy_sum != vap_volume_sum:
    print('error')
else:
    print('OK')


In [ ]:
#　合計値がOHLCVとVAPで一致することを確認する。
ohlcvv = market.ohlcvv(start_time=0, end_time=0, window_sec=60)

sum = ohlcvv['volume'].sum()

sum1 = ohlcv['volume'].sum()
sum2 = vap['buy_volume'].sum() + vap['sell_volume'].sum()

print(f'OHLCVVの合計値: {sum}')
print(f'OHLCVの合計値: {sum1}')
print(f'VAPの合計値: {sum2}')
